## 데이터 불러오기

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('titanic_processed.csv')

In [3]:
df

,age,sibsp,parch,fare,pclass_1.0,pclass_2.0,pclass_3.0,sex_female,embarked_C,embarked_Q,embarked_S,survived
0,29.0000,0.0,0.0,211.3375,1,0,0,1,0,0,1,1
1,0.9167,1.0,2.0,151.5500,1,0,0,0,0,0,1,1
2,2.0000,1.0,2.0,151.5500,1,0,0,1,0,0,1,0
3,30.0000,1.0,2.0,151.5500,1,0,0,0,0,0,1,0
4,25.0000,1.0,2.0,151.5500,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1038,45.5000,0.0,0.0,7.2250,0,0,1,0,1,0,0,0
1039,14.5000,1.0,0.0,14.4542,0,0,1,1,1,0,0,0
1040,26.5000,0.0,0.0,7.2250,0,0,1,0,1,0,0,0
1041,27.0000,0.0,0.0,7.2250,0,0,1,0,1,0,0,0


In [4]:
X=df[df.columns[:-1]]
y=df['survived']

## 다항식 추가에 따른 효과 검증

In [7]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import PolynomialFeatures

## 2차항 및 교호작용 추가
transformer = PolynomialFeatures(degree=2, interaction_only=False)
X_=transformer.fit_transform(X)
X_

array([[ 1.    , 29.    ,  0.    , ...,  0.    ,  0.    ,  1.    ],
       [ 1.    ,  0.9167,  1.    , ...,  0.    ,  0.    ,  1.    ],
       [ 1.    ,  2.    ,  1.    , ...,  0.    ,  0.    ,  1.    ],
       ...,
       [ 1.    , 26.5   ,  0.    , ...,  0.    ,  0.    ,  0.    ],
       [ 1.    , 27.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
       [ 1.    , 29.    ,  0.    , ...,  0.    ,  0.    ,  1.    ]])

## 2차항 추가된 모형의 교차검증

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_validate

## Kfold 검증
kfold = KFold(5,shuffle=False)
reg=LogisticRegression()
result = cross_validate(estimator=reg, X=X_, y=y, cv=kfold, scoring=['accuracy','roc_auc'], return_train_score=True)
print(result['test_accuracy'].mean(), result['test_roc_auc'].mean())

0.7784412955465587 0.7986088445710039


In [14]:
kfold = KFold(5,shuffle=False)
for i in range(1,6):
    transformer = PolynomialFeatures(degree=i, interaction_only=False)
    X_=transformer.fit_transform(X)
    reg=LogisticRegression()
    result = cross_validate(estimator=reg, X=X_, y=y, cv=kfold, scoring=['accuracy','roc_auc'], return_train_score=True)
    print(i, 'train', result['train_accuracy'].mean(), result['train_roc_auc'].mean())
    print(i, 'test', result['test_accuracy'].mean(), result['test_roc_auc'].mean())

1 train 0.7955381323683568 0.8501926535399121
1 test 0.7841783216783218 0.8145760274935897
2 train 0.7988942977354643 0.8558156425741796
2 test 0.7784412955465587 0.7986088445710039
3 train 0.7835549045793305 0.8152820005689035
3 test 0.7554793890320206 0.759272957457617
4 train 0.7358643863352432 0.7531575631765879
4 test 0.7325220831799779 0.7014473399030657
5 train 0.5925400996568015 0.5
5 test 0.5928183658446816 0.5


## 모델규제

In [28]:
kfold = KFold(5,shuffle=False)
for i in range(1,6):
    transformer = PolynomialFeatures(degree=i, interaction_only=False)
    X_=transformer.fit_transform(X)
    reg=LogisticRegression(penalty='l2',C=0.001)
    result = cross_validate(estimator=reg, X=X_, y=y, cv=kfold, scoring=['accuracy','roc_auc'], return_train_score=True)
    print(i, 'train', result['train_accuracy'].mean(), result['train_roc_auc'].mean())
    print(i, 'test', result['test_accuracy'].mean(), result['test_roc_auc'].mean())

1 train 0.6713870101523571 0.7202838614377776
1 test 0.6252576370997425 0.6757365126929544
2 train 0.8058441390600095 0.8567908965603952
2 test 0.7842013249907986 0.7992671818959173
3 train 0.779003431984951 0.8132483973709803
3 test 0.7574116672800884 0.7589459844289624
4 train 0.7305877453725642 0.7473374797919211
4 test 0.7229435038645565 0.700712925067241
5 train 0.5925400996568015 0.5
5 test 0.5928183658446816 0.5
